# Welcome!
In this tutorial we'll walk through a simple convolutional neural network to classify the images in the cifar10 dataset.

We’ll also set up Weights & Biases to log models metrics, inspect performance and share findings about the best architecture for the network. In this example we're using Google Colab as a convenient hosted environment, but you can run your own training scripts from anywhere and visualize metrics with W&B's experiment tracking tool.

### Running This Notebook
1. Click "Open in playground" to create a copy of this notebook for yourself.
2. Save a copy in Google Drive for yourself.
3. To enable a GPU, please click Edit > Notebook Settings. Change the "hardware accelerator" to GPU.
4. Step through each section below, pressing play on the code blocks to run the cells.

Results will be logged to a [shared W&B project page](https://app.wandb.ai/wandb/cnn-intro).

We highly encourage you to fork this notebook, tweak the parameters, or try the model with your own dataset!

# Convolutional Neural Networks
**[Slides](https://www.dropbox.com/s/hhb2ohd8wdpccnr/Bloomberg%20Class%202.pdf?dl=0)**

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1573598995521_Screenshot+2019-11-12+14.49.51.png)

**Convolution Layer** The convolution layer is made up of a set of independent filters. Each filter slides over the image and creates feature maps that learn different aspects of an image.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1573598824588_Screenshot+2019-11-12+14.46.40.png)

**Convolutional Neural Network**
A CNN uses convolutions to connected extract features from local regions of an input. Most CNNs contain a combination of convolutional, pooling and affine layers. CNNs offer fantastic performance on visual recognition tasks, where they have become the state of the art.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1573598824566_Screenshot+2019-11-12+14.46.59.png)

**Pooling** The pooling layer reduce the size of the image representation and so the number of parameters and computation in the network. Pooling usually involves taking either the maximum or average value across the pooled area.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1573598953569_Screenshot+2019-11-12+14.49.10.png)

### Basic Convolutional Neural Network

In [0]:
# Install wandb
!pip install -qq wandb
#import libraries
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
os.environ['WANDB_NOTEBOOK_NAME'] = 'cnn-intro'

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 460kB 40.5MB/s 
     |████████████████████████████████| 256kB 47.0MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 184kB 44.7MB/s 


In [0]:
import tensorflow as tf
import wandb

# Set Hyper-parameters
wandb.init(entity="wandb", project="cnn-intro")
config = wandb.config
config.concept = 'cnn'
config.batch_size = 128
config.epochs = 10
config.learn_rate = 0.001
config.dropout = 0.3
config.dense_layer_nodes = 128



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [0]:
%%wandb
# Load data
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(class_names)
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# normalize data
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define model
model = tf.keras.models.Sequential()
# Conv2D adds a convolution layer with 32 filters that generates 2 dimensional
# feature maps to learn different aspects of our image
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',
                    input_shape=X_train.shape[1:], activation='relu'))

# MaxPooling2D layer reduces the size of the image representation our
# convolutional layers learnt, and in doing so it reduces the number of
# parameters and computations the network needs to perform.
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Dropout layer turns off a percentage of neurons at every step
model.add(tf.keras.layers.Dropout(config.dropout))

# Flattens our array so we can feed the convolution layer outputs (a matrix)
# into our fully connected layer (an array)
model.add(tf.keras.layers.Flatten())

# Dense layer creates dense, fully connected layers with x inputs and y outputs
# - it simply outputs the dot product of our inputs and weights
model.add(tf.keras.layers.Dense(config.dense_layer_nodes, activation='relu'))
model.add(tf.keras.layers.Dropout(config.dropout))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile the model and specify the optimizer and loss function
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(config.learn_rate),
              metrics=['accuracy'])

# log the number of total parameters
config.total_params = model.count_params()
print("Total params: ", config.total_params)

# Fit the model to the training data, specify the batch size
# and the WandbCallback() to track model
model.fit(X_train, y_train, epochs=10, batch_size=128,
          validation_data=(X_test, y_test), verbose=0,
          callbacks=[wandb.keras.WandbCallback(data_type="image",
          labels=class_names, save_model=False)])

170500096/170498071 [==============================] - 11s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Total params:  1050890


### Convolutional Neural Network with Data Augmentation
Data augmentation artificially expands the training dataset by creating slightly modified versions of images in the dataset - by scaling, shifting and rotating the images in the training set.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1573599686286_Screenshot+2019-11-12+15.01.23.png)

In [0]:
# examples/keras-cifar/cifar-gen.py
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import os
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf

# Define hyperparameters
run = wandb.init(entity="wandb", project="cnn-intro")
config = run.config
config.concept = 'cnn-augmented'
config.dropout = 0.25
config.dense_layer_nodes = 100
config.learn_rate = 0.08
config.batch_size = 128
config.epochs = 10

# Load data
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(class_names)

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define the model (same as above)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',
                                 input_shape=X_train.shape[1:], activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(config.dropout))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(config.dense_layer_nodes, activation='relu'))
model.add(tf.keras.layers.Dropout(config.dropout))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# log the number of total parameters
config.total_params = model.count_params()
print("Total params: ", config.total_params)

# normalize data
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Add data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow()
model.fit_generator(datagen.flow(X_train, y_train,
                                 batch_size=config.batch_size),
                    steps_per_epoch=X_train.shape[0] // config.batch_size,
                    epochs=config.epochs, verbose =0,
                    validation_data=(X_test, y_test),
                    callbacks=[WandbCallback(data_type="image", labels=class_names)])

Total params:  821206


# Visualize Predictions Live

## Project Overview
1. Check out the [project page](https://app.wandb.ai/wandb/cnn-intro) to see your results in the shared project. 
1. Press 'option+space' to expand the runs table, comparing all the results from everyone who has tried this script. 
1. Click on the name of a run to dive in deeper to that single run on its own run page.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1574117851346_Screenshot+2019-11-18+14.57.27.png)

## Visualize Performance
Click through to a single run to see more details about that run. For example, on [this run page](https://app.wandb.ai/wandb/cnn-intro/runs/6218o47i) you can see the performance metrics I logged when I ran this script.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1574117639961_Screenshot+2019-11-18+14.53.27.png)


## Review Code
The overview tab picks up a link to the code. In this case, it's a link to the Google Colab. If you're running a script from a git repo, we'll pick up the SHA of the latest git commit and give you a link to that version of the code in your own GitHub repo.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1574117783115_overview.png)

## Visualize System Metrics
The System tab on the runs page lets you visualize how resource efficient your model was. It lets you monitor the GPU, memory, CPU, disk, and network usage in one spot.

![](https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1574118253358_Screenshot+2019-11-18+15.04.10.png)


## Next Steps
As you can see running sweeps is super easy! We highly encourage you to fork this notebook, tweak the parameters, or try the model with your own dataset!


## More about Weights & Biases
We're always free for academics and open source projects. Email carey@wandb.com with any questions or feature suggestions. Here are some more resources:

1. [Documentation](http://docs.wandb.com) - Python docs
2. [Gallery](https://app.wandb.ai/gallery) - example reports in W&B
3. [Articles](https://www.wandb.com/articles) - blog posts and tutorials
4. [Community](bit.ly/wandb-forum) - join our Slack community forum